In [57]:
from pydub import AudioSegment

def detect_leading_silence(sound, silence_threshold=-30.0, chunk_size=10):
    '''
    sound is a pydub.AudioSegment
    silence_threshold in dB
    chunk_size in ms

    iterate over chunks until you find the first one with sound
    '''
    trim_ms = 0 # ms

    assert chunk_size > 0 # to avoid infinite loop
    while sound[trim_ms:trim_ms+chunk_size].dBFS < silence_threshold and trim_ms < len(sound):
        trim_ms += chunk_size

    return trim_ms

sound = AudioSegment.from_file("/Users/apple/Downloads/Audio/Development/Freeform/206_1_Freeform_audio.mp4", format="mp4")
print(type(sound))

start_trim = detect_leading_silence(sound)
end_trim = detect_leading_silence(sound.reverse())

duration = len(sound)    
trimmed_sound = sound[start_trim:duration-end_trim]
trimmed_sound.export('test.mp4', format="mp4")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/apple/Downloads/Audio/Development/Freeform/206_1_Freeform_audio.mp4'

In [58]:
import os

def del_sil_in_batch(path, save_path):
    file_list = os.listdir(path)
    file_list.sort()
    if '.DS_Store' in file_list:
        file_list.remove('.DS_Store')
    if '.mp4' in file_list:
        file_list.remove('.mp4')
    for file in file_list:
        if os.path.isfile(os.path.join(path, file)):
            sound = AudioSegment.from_file(os.path.join(path, file), format="wav")
            start_trim = detect_leading_silence(sound)
            end_trim = detect_leading_silence(sound.reverse())
            duration = len(sound)    
            trimmed_sound = sound[start_trim:duration-end_trim]
            trimmed_sound.export(os.path.join(save_path, file[:5])+'.wav', format="wav")
#             sound.export(os.path.join(save_path, file[:5])+'.wav', format="wav")
                        
paths = ['/Users/apple/Downloads/depression/AViD/Audio/Development/Freeform', '/Users/apple/Downloads/depression/AViD/Audio/Training/Freeform', '/Users/apple/Downloads/depression/AViD/Audio/Testing/Freeform']
save_paths = ['/Users/apple/Downloads/depression/AViD/Audio/Development/Trim', '/Users/apple/Downloads/depression/AViD/Audio/Training/Trim', '/Users/apple/Downloads/depression/AViD/Audio/Testing/Trim']

for i in range(len(paths)):
    del_sil_in_batch(paths[i], save_paths[i])
            

In [43]:
import numpy as np
import pickle

def generate_labels(path):
    file_list = os.listdir(path)
    file_list.sort()
    if '.DS_Store' in file_list:
        file_list.remove('.DS_Store')
    nums, labels = [], []
    for file in file_list:
        if os.path.isfile(os.path.join(path, file)):
            nums.append(file[:5])
            with open(os.path.join(path, file)) as f:
                labels.append(f.readlines()[0].split('\n')[0])
    return [nums, labels]

paths = ['/Users/apple/Downloads/depression/AViD/AVEC2014_DepressionLabels/Development_DepressionLabels',
         '/Users/apple/Downloads/depression/AViD/AVEC2014_DepressionLabels/Testing_DepressionLabels',
         '/Users/apple/Downloads/depression/AViD/AVEC2014_DepressionLabels/Training_DepressionLabels']

info = {}
info['dev'] = generate_labels(paths[0])
info['train'] = generate_labels(paths[2])
info['test'] = generate_labels(paths[1])
test_index = list(set(info['test'][0])-set(info['dev'][0])-set(info['train'][0]))
new_test_labels = []
path = paths[1]
for file in test_index:
    print(os.path.isfile(os.path.join(path, file)+'_Depression.csv'), os.path.join(path, file)+'_Depression.csv')
    if os.path.isfile(os.path.join(path, file)+'_Depression.csv'):
        with open(os.path.join(path, file)+'_Depression.csv') as f:
            new_test_labels.append(f.readlines()[0].split('\n')[0])
info['test'][0] = test_index
info['test'][1] = new_test_labels

print(info['train'])

# np.save('avid_info.npy', info)
with open('avid_info.pkl', 'wb') as f:
    pickle.dump(info, f)

True /Users/apple/Downloads/depression/AViD/AVEC2014_DepressionLabels/Testing_DepressionLabels/310_2_Depression.csv
True /Users/apple/Downloads/depression/AViD/AVEC2014_DepressionLabels/Testing_DepressionLabels/315_3_Depression.csv
True /Users/apple/Downloads/depression/AViD/AVEC2014_DepressionLabels/Testing_DepressionLabels/344_2_Depression.csv
True /Users/apple/Downloads/depression/AViD/AVEC2014_DepressionLabels/Testing_DepressionLabels/365_1_Depression.csv
True /Users/apple/Downloads/depression/AViD/AVEC2014_DepressionLabels/Testing_DepressionLabels/246_1_Depression.csv
True /Users/apple/Downloads/depression/AViD/AVEC2014_DepressionLabels/Testing_DepressionLabels/335_2_Depression.csv
True /Users/apple/Downloads/depression/AViD/AVEC2014_DepressionLabels/Testing_DepressionLabels/325_2_Depression.csv
True /Users/apple/Downloads/depression/AViD/AVEC2014_DepressionLabels/Testing_DepressionLabels/245_3_Depression.csv
True /Users/apple/Downloads/depression/AViD/AVEC2014_DepressionLabels/Te

In [59]:
import os

def del_sil_in_batch(path, save_path):
    file_list = os.listdir(path)
    file_list.sort()
    if '.DS_Store' in file_list:
        file_list.remove('.DS_Store')
    if '.mp4' in file_list:
        file_list.remove('.mp4')
    for file in file_list:
        if os.path.isfile(os.path.join(path, file)):
            try:
                sound = AudioSegment.from_file(os.path.join(path, file), format="wav")
            except:
                print(os.path.join(path, file))
                continue
            sound.export(os.path.join(save_path, file), format="wav", bitrate='441k')
                        
paths = ['/Users/apple/Downloads/depression/AViD/Audio/Development/Trim', '/Users/apple/Downloads/depression/AViD/Audio/Training/Trim', '/Users/apple/Downloads/depression/AViD/Audio/Testing/Trim']
save_paths = ['/Users/apple/Downloads/depression/AViD/Audio/Development/Trim', '/Users/apple/Downloads/depression/AViD/Audio/Training/Trim', '/Users/apple/Downloads/depression/AViD/Audio/Testing/Trim']

for i in range(len(paths)):
    del_sil_in_batch(paths[i], save_paths[i])
            

/Users/apple/Downloads/depression/AViD/Audio/Development/Trim/241_1.wav
/Users/apple/Downloads/depression/AViD/Audio/Development/Trim/241_3.wav
/Users/apple/Downloads/depression/AViD/Audio/Training/Trim/234_3.wav
/Users/apple/Downloads/depression/AViD/Audio/Training/Trim/238_2.wav
/Users/apple/Downloads/depression/AViD/Audio/Training/Trim/241_2.wav
/Users/apple/Downloads/depression/AViD/Audio/Training/Trim/308_3.wav
/Users/apple/Downloads/depression/AViD/Audio/Testing/Trim/310_2.wav


In [61]:
import sox

def upsample_wav(path, save_path):
    file_list = os.listdir(path)
    file_list.sort()
    if '.DS_Store' in file_list:
        file_list.remove('.DS_Store')
    if '.mp4' in file_list:
        file_list.remove('.mp4')
    for file in file_list:
        if os.path.isfile(os.path.join(path, file)):
            tfm = sox.Transformer()
            tfm.rate(44100)
            try:
                tfm.build(os.path.join(path, file), os.path.join(save_path, file))
            except:
                print(os.path.join(path, file))
                continue
    
paths = ['/Users/apple/Downloads/depression/AViD/Audio/Development/Trim', '/Users/apple/Downloads/depression/AViD/Audio/Training/Trim', '/Users/apple/Downloads/depression/AViD/Audio/Testing/Trim']
save_paths = ['/Users/apple/Downloads/depression/AViD/Audio/Development/tmp', '/Users/apple/Downloads/depression/AViD/Audio/Training/tmp', '/Users/apple/Downloads/depression/AViD/Audio/Testing/tmp']

for i in range(len(paths)):
    upsample_wav(paths[i], save_paths[i])
    

In [ ]:
def del_copies(path, save_path):
    file_list = os.listdir(path)
    file_list.sort()
    if '.DS_Store' in file_list:
        file_list.remove('.DS_Store')
    if '.mp4' in file_list:
        file_list.remove('.mp4')
    for file in file_list:
        if os.path.isfile(os.path.join(path, file)):
            tfm = sox.Transformer()
            tfm.rate(44100)
            try:
                tfm.build(os.path.join(path, file), os.path.join(save_path, file))
            except:
                print(os.path.join(path, file))
                continue
    
paths = ['/Users/apple/Downloads/depression/AViD/Audio/Development/Trim', '/Users/apple/Downloads/depression/AViD/Audio/Training/Trim', '/Users/apple/Downloads/depression/AViD/Audio/Testing/Trim']
save_paths = ['/Users/apple/Downloads/depression/AViD/Audio/Development/new_trim', '/Users/apple/Downloads/depression/AViD/Audio/Training/new_trim', '/Users/apple/Downloads/depression/AViD/Audio/Testing/new_trim']

for i in range(len(paths)):
    upsample_wav(paths[i], save_paths[i])
    